In [14]:
import pandas as pd
from functions import trim_and_lower, extract_string_in_parentheses, drop_non_numbers
import re
import yaml

In [2]:
!ls ..

config.yaml          requirements-dev.in  slides
data                 requirements-dev.txt venv
images               requirements.in
notebooks            requirements.txt


In [15]:
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("The configuration file was not found!")

In [16]:
data_dict_path = config['data']['raw']['raw_5']
data_dict_df = pd.read_csv(data_dict_path)
data_dict_df.head()

,Table,Field,Description
0,Tours,Year,Year of the race
1,Tours,Dates,Dates in which the race took place
2,Tours,Stages,Number of stages in the race
3,Tours,Distance,Total race distance in kilometers (and miles)
4,Tours,Starters,Number of cyclists that started the race


In [6]:
#path = "../data/raw/tdf_tours.csv"
df = pd.read_csv(config["data"]["raw"]["raw_3"], encoding='cp1252')
df

,Year,Dates,Stages,Distance,Starters,Finishers
0,1903,1–19 July 1903,6,"2,428 km (1,509 mi)",60,21
1,1904,2–24 July 1904,6,"2,428 km (1,509 mi)",88,15
2,1905,9–30 July 1905,11,"2,994 km (1,860 mi)",60,24
3,1906,4–29 July 1906,13,"4,637 km (2,881 mi)",82,14
4,1907,8 July – 4 August 1907,14,"4,488 km (2,789 mi)",93,33
...,...,...,...,...,...,...
104,2018,7–29 July 2018,21,"3,349 km (2,081 mi)",176,145
105,2019,6–28 July 2019,21,"3,366 km (2,092 mi)",176,155
106,2020,29 August — 20 September 2020[1],21,"3,484 km (2,165 mi)",176,146
107,2021,26 June – 18 July 2021,21,"3,414.4 km (2,121.6 mi)",184,141


In [7]:
df = trim_and_lower(df)
df.head()

,year,dates,stages,distance,starters,finishers
0,1903,1–19 july 1903,6,"2,428 km (1,509 mi)",60,21
1,1904,2–24 july 1904,6,"2,428 km (1,509 mi)",88,15
2,1905,9–30 july 1905,11,"2,994 km (1,860 mi)",60,24
3,1906,4–29 july 1906,13,"4,637 km (2,881 mi)",82,14
4,1907,8 july – 4 august 1907,14,"4,488 km (2,789 mi)",93,33


In [8]:
df.stages.isna().sum()

0

In [9]:
#df["miles"] = df.distance.apply([lambda x: pd.Series(extract_string_in_parentheses(x))]).apply([lambda x: pd.Series(drop_non_numbers(x))])

#df["miles"] = df.distance.apply([lambda x: pd.Series(drop_non_numbers(extract_string_in_parentheses(x)))])
df["miles"] = df.distance.apply(lambda x: drop_non_numbers(extract_string_in_parentheses(x)))
df["km"] = df.distance.apply(lambda x: drop_non_numbers(x.split("(")[0]))
df.stages = df.stages.apply(lambda x: int(drop_non_numbers(x)))
#df.stages = df.stages.apply(lambda x: int(x))
df.head()

,year,dates,stages,distance,starters,finishers,miles,km
0,1903,1–19 july 1903,6,"2,428 km (1,509 mi)",60,21,1509.0,2428.0
1,1904,2–24 july 1904,6,"2,428 km (1,509 mi)",88,15,1509.0,2428.0
2,1905,9–30 july 1905,11,"2,994 km (1,860 mi)",60,24,1860.0,2994.0
3,1906,4–29 july 1906,13,"4,637 km (2,881 mi)",82,14,2881.0,4637.0
4,1907,8 july – 4 august 1907,14,"4,488 km (2,789 mi)",93,33,2789.0,4488.0


In [10]:
cols_to_drop = [
    "distance",
    "dates"
]

df = df.drop(columns=cols_to_drop)
df.head()

,year,stages,starters,finishers,miles,km
0,1903,6,60,21,1509.0,2428.0
1,1904,6,88,15,1509.0,2428.0
2,1905,11,60,24,1860.0,2994.0
3,1906,13,82,14,2881.0,4637.0
4,1907,14,93,33,2789.0,4488.0


In [11]:
df.dtypes

year           int64
stages         int64
starters       int64
finishers      int64
miles        float64
km           float64
dtype: object

In [12]:
df.shape

(109, 6)

In [13]:
# Save the DataFrame to a new CSV file with 'utf-8' encoding
#output_file_path = "/Users/anthonydillon/Ironhack/wk3/project/data/clean/tours.csv"
output_file_path = config['data']['clean']['clean_3']
df.to_csv(output_file_path, encoding='utf-8', index=False)